---
# Задание 3. Применение RNN для прогнозирования отношения в рецензиях на фильмы.

- Реализовать многослойную рекуррентную нейронную сеть для смыслового анализа рецензий на фильмы IMDb. ✅

- Использовать пакетное обучение RNN. ✅

- Применять слой LSTM для учета долгосрочных эффектов. Поместить слой LSTM внутрь оболочки Bidirectional. ✅

### Отчетность
Отчет по заданию должен быть оформлен в виде ноутбука с прокомментированным кодом и результатами запуска кода. Ноутбук с отчетом прикрепите к странице до 31 мая (включительно).

---
### Подключение модулей и библиотек

In [1]:
from collections import Counter

import numpy as np
import os as os
import pandas as pd
import pickle
from html.parser import HTMLParser

import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers

C:\Users\Artyom\anaconda3\envs\nlp-env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


---
### План подготовки данных

1. Создать объект набора данных TensorFlow и расщепить его на обучающую, испытательную и проверочную части;
2. Идентифицировать уникальные слова в обучающем наборе;
3. Отобразить каждое уникальное сллово на уникальное целое число и закодировать текст рецензии с помощью этих целых чисел (индексов уникальных слов);
4. Разделить набор данных на мини-пакеты, которые будут служить входом для модели.

---
### Загрузка датасета
Загрузим датасет, подготовленный при выполнении предыдущего задания. Он содержит два столбца: 
- **review_text:**  исходный текст рецензии;
- **sentiment_label:**  метка; 

In [2]:
df = pd.read_pickle('df.pickle')

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   review_text      50000 non-null  object
 1   sentiment_label  50000 non-null  int8  
dtypes: int8(1), object(1)
memory usage: 439.6+ KB


In [4]:
df.head(3)

,review_text,sentiment_label
0,Bromwell High is a cartoon comedy. It ran at t...,1
1,Homelessness (or Houselessness as George Carli...,1
2,Brilliant over-acting by Lesley Ann Warren. Be...,1


---
### Подготовка данных

In [5]:
data = df.review_text
target = df.sentiment_label

Шаг 1: сформируем tensorflow-датасет.

In [6]:
# Создание набора данных
ds_raw = tf.data.Dataset.from_tensor_slices((data.values, target.values))

# Инспектирование
for example in ds_raw.take(3):
    tf.print(example[0].numpy()[:50], example[1])

b'Bromwell High is a cartoon comedy. It ran at the s' 1
b'Homelessness (or Houselessness as George Carlin st' 1
b'Brilliant over-acting by Lesley Ann Warren. Best d' 1


In [7]:
tf.random.set_seed(2023)
ds_raw = ds_raw.shuffle(50000, reshuffle_each_iteration=False)

ds_raw_test = ds_raw.take(25000)
ds_raw_train_val = ds_raw.skip(25000)
ds_raw_train = ds_raw_train_val.take(20000)
ds_raw_val = ds_raw_train_val.skip(20000)

Шаг 2: найдём уникальные лексемы с помощью `Counter`.

In [8]:
%%time

tokenizer = tfds.features.text.Tokenizer()
token_counts = Counter()

for example in ds_raw_train:
    tokens = tokenizer.tokenize(example[0].numpy())
    token_counts.update(tokens)
    
print(f'Размер словаря: {len(token_counts)}')

Размер словаря: 86808
CPU times: total: 7.53 s
Wall time: 8.79 s


Шаг 3: закодируем уникальные лексемы целыми числами с помощью `tfds.features.text.TokenTextEncoder`.

In [9]:
encoder = tfds.features.text.TokenTextEncoder(token_counts)
example_str = 'This is an example!'
print(encoder.encode(example_str))

[810, 57, 177, 3189]


Определим вспомогательные функции:

In [10]:
def encode(text_tensor, label):
    text = text_tensor.numpy()
    encoded_text = encoder.encode(text)
    return encoded_text, label


def encode_map(text, label):
    return tf.py_function(encode, inp=[text, label],
                          Tout=(tf.int64, tf.int8))

In [11]:
ds_train = ds_raw_train.map(encode_map)
ds_val = ds_raw_val.map(encode_map)
ds_test = ds_raw_test.map(encode_map)

Выведем длины нескольких примеров из обучающей выборки:

In [12]:
tf.random.set_seed(2023)

for example in ds_train.shuffle(1000).take(5):
    print(f'Длина последовательности: {example[0].shape}')

Длина последовательности: (459,)
Длина последовательности: (235,)
Длина последовательности: (165,)
Длина последовательности: (151,)
Длина последовательности: (223,)


Разделим наборы на пакеты:

In [13]:
train_data = ds_train.padded_batch(32, padded_shapes=([-1],[]))
val_data = ds_val.padded_batch(32, padded_shapes=([-1],[]))
test_data = ds_test.padded_batch(32, padded_shapes=([-1],[]))

---
### Разработка модели

In [14]:
embedding_dim=20
vocab_size = len(token_counts) + 2

tf.random.set_seed(2023)

# построение модели
bi_lstm_model = tf.keras.Sequential([
    layers.Embedding(input_dim=vocab_size,
                     output_dim=embedding_dim,
                     name='embed_layer'), 
    layers.Bidirectional(
        layers.LSTM(64, name='lstm_layer'),
        name='bidir-lstm'),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')
])

bi_lstm_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embed_layer (Embedding)     (None, None, 20)          1736200   
                                                                 
 bidir-lstm (Bidirectional)  (None, 128)               43520     
                                                                 
 dense (Dense)               (None, 64)                8256      
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 1,788,041
Trainable params: 1,788,041
Non-trainable params: 0
_________________________________________________________________


Скомпилируем и обучим модель:

In [15]:
bi_lstm_model.compile(
    optimizer=tf.keras.optimizers.Adam(1e-3),
    loss=tf.keras.losses.BinaryCrossentropy(from_logits=False),
    metrics=['accuracy']
)

В целях экономии времени попробуем обучить модель всего на двух эпохах:

In [16]:
history = bi_lstm_model.fit(
    train_data,
    validation_data=val_data,
    epochs=2,
    use_multiprocessing=True
)

Epoch 1/2
625/625 [==============================] - 383s 604ms/step - loss: 0.4956 - accuracy: 0.7617 - val_loss: 0.3822 - val_accuracy: 0.8486
Epoch 2/2
625/625 [==============================] - 365s 584ms/step - loss: 0.2465 - accuracy: 0.9082 - val_loss: 0.3818 - val_accuracy: 0.8562


Проведём оценку построенной модели:

In [17]:
test_results = bi_lstm_model.evaluate(test_data)
print(f'Правильность при испытании: {test_results[1]}')

782/782 [==============================] - 91s 117ms/step - loss: 0.3613 - accuracy: 0.8642
Правильность при испытании: 0.8641600012779236
